In [1]:
from __future__ import division
import os
import gzip
import tarfile
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as ss
import matplotlib.image as img
import tensorflow as tf
from sklearn.utils import shuffle
import matplotlib.image as mpimg

/docs/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
"""loading emotion labels from file"""
lables_y = np.load('labels.npy')
#print(lables_y)
"""loading images - pixel matrices from file"""
images = np.load('image_matrix.npy')
images, lables_y = shuffle(images, lables_y, random_state=0)

In [3]:
"""splitting to train and test"""
train_images = images[0:300]
#print(train_images.shape)
test_images = images[300:327]
#train_y = train_labels_arr.reshape(300,1)
#test_y = test_labels_arr.reshape(27,1)
train_y = lables_y[0:300]
test_y = lables_y[300: 327]
#print(train_y)
"""normalization"""
train_x_norm = (train_images / 255)
test_x_norm = (test_images / 255)

In [4]:
IMAGE_SIZE = 256

def central_scale_images(X_imgs, y_images, scales):
    # Various settings needed for Tensorflow operation
    boxes = np.zeros((len(scales), 4), dtype=np.float32)
    for index, scale in enumerate(scales):
        x1 = y1 = 0.5 - 0.5 * scale  # To scale centrally
        x2 = y2 = 0.5 + 0.5 * scale
        boxes[index] = np.array([y1, x1, y2, x2], dtype=np.float32)
    box_ind = np.zeros((len(scales)), dtype=np.int32)
    crop_size = np.array([IMAGE_SIZE, IMAGE_SIZE], dtype=np.int32)
    X_scale_data = []
    y_scale_data = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape=(1, IMAGE_SIZE, IMAGE_SIZE, 1))
    # Define Tensorflow operation for all scales but only one base image at a time
    tf_img = tf.image.crop_and_resize(X, boxes, box_ind, crop_size)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        counter = 0
        for img_data in X_imgs:
            # a,b,c = np.where(a == img_data)
            batch_img = np.expand_dims(img_data, axis=0)
            scaled_imgs = sess.run(tf_img, feed_dict={X: batch_img})

            X_scale_data.extend(scaled_imgs)
            for i in range(3):
                y_scale_data.append(y_images[counter])
            counter = counter + 1
    X_scale_data = np.array(X_scale_data, dtype=np.float32)
    y_scale_data = np.array(y_scale_data, dtype=np.float32)

    return X_scale_data, y_scale_data

In [5]:
def flip_images(X_imgs, y_images):
    X_flip = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape = (IMAGE_SIZE, IMAGE_SIZE, 1))
    tf_img1 = tf.image.flip_left_right(X)
    tf_img2 = tf.image.flip_up_down(X)
    tf_img3 = tf.image.transpose_image(X)
    counter = 0
    y_flip= []
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for img in X_imgs:
            flipped_imgs = sess.run([tf_img1, tf_img2, tf_img3], feed_dict = {X: img})
            X_flip.extend(flipped_imgs)
            for i in range(3):
                y_flip.append(y_images[counter])
            counter = counter + 1
    X_flip = np.array(X_flip, dtype = np.float32)
    y_flip = np.array(y_flip, dtype=np.float32)
    return X_flip, y_flip

In [6]:
train_x_norm1 = train_x_norm.reshape(train_x_norm.shape[0], train_x_norm.shape[1], train_x_norm.shape[2], 1)
test_x_norm1 = test_x_norm.reshape(test_x_norm.shape[0], test_x_norm.shape[1], test_x_norm.shape[2], 1)

# Produce each image at scaling of 90%, 75% and 60% of original image.

X_imgs_train, y_imgs_train = central_scale_images(train_x_norm1 , lables_y, [0.90, 0.75, 0.60])
X_flipped_train,y_flipped_train = flip_images(train_x_norm1 , lables_y)

train_X = np.concatenate((X_imgs_train , X_flipped_train), axis=0)
train_Y = np.concatenate((y_imgs_train, y_flipped_train), axis=0)
print(train_X.shape)

(1800, 256, 256, 1)


In [7]:
"""reshaping X to contain channels = 1"""
#print(train_x_norm.shape)
train_x_norm1 = train_x_norm.reshape(train_x_norm.shape[0], train_x_norm.shape[1], train_x_norm.shape[2], 1)
test_x_norm1 = test_x_norm.reshape(test_x_norm.shape[0], test_x_norm.shape[1], test_x_norm.shape[2], 1)

In [8]:
"""one hot encoding function"""
def One_Hot_Encoding(arr, samples_num):
    encode_matrix = np.zeros((samples_num, 8))
    for i in range(samples_num):
        #print(type(arr[i][0]))
        encode_matrix[i][int(float(arr[i][0]))] = 1
    return encode_matrix


train_y_encode = One_Hot_Encoding(train_Y, len(train_Y))
test_y_encode = One_Hot_Encoding(test_y, len(test_y))

In [9]:
"""function to initialize weights and bias for convolution 2 layers"""
def initialize_weights():
    
    tf.set_random_seed(1)                              
        
    W1 = tf.get_variable('W1',[5, 5, 1, 6], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W2 = tf.get_variable('W2',[5, 5, 6, 8], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W3 = tf.get_variable('W3',[5, 5, 8, 10], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
   # B1 = tf.get_variable('B1', tf.zeros([8], tf.float32), name=)
    B1 = tf.get_variable('B1', [6], initializer=tf.zeros_initializer())
    B2 = tf.get_variable('B2', [8], initializer=tf.zeros_initializer())
    B3 = tf.get_variable('B3', [10], initializer=tf.zeros_initializer())
   # B2 = tf.zeros([12], tf.float32)
    #B3 = tf.zeros([16], tf.float32)
    weights = ( W1, W2, W3)
    bias = ( B1,B2, B3)
    #print("weights intialized")
    #print(W1)
    return weights, bias

In [10]:
"""forward propagation. convolution and fully connected. dropout is applied as regularization for fully conencted layers"""


def frwd_propagation(X_input, weights, bias):
      
    
    W1, W2, W3 = weights
    B1, B2, B3 = bias
    # first convolution with relu as activation
    conv1 = tf.nn.conv2d(X_input, W1, strides = [1,1,1,1], padding = 'SAME', name='conv1')
    print("conv1 layer shape")
    print(conv1.shape)
    Z1 = tf.nn.bias_add(conv1, B1)
    A1 = tf.nn.relu(Z1)
    #max pooling
    pool1 = tf.nn.max_pool(A1, ksize = [1,3,3,1], strides = [1,2,2,1], padding = 'VALID')
    print("pool1 shape")
    print(pool1.shape)
    # 2nd convolution layer with relu as activation
    conv2 = tf.nn.conv2d(pool1, W2, strides = [1,1,1,1], padding = 'VALID', name='conv2')
    Z2 = tf.nn.bias_add(conv2, B2)
    A2 = tf.nn.relu(Z2)
    print("conv2 shape")
    print(conv2.shape)
    #avg pooling
    pool2 = tf.nn.max_pool(A2, ksize = [1,3,3,1], strides = [1,2,2,1], padding = 'VALID')
    #pool2 = tf.nn.avg_pool(A2, ksize = [1,8,8,1], strides = [1,1,1,1], padding = 'VALID', data_format='NHWC', name=None)
    print("pool2 shape before unfaltten")
    print(pool2.shape)
    conv3 = tf.nn.conv2d(pool2, W3, strides = [1,1,1,1], padding = 'SAME', name='conv3')
    Z3 = tf.nn.bias_add(conv3, B3)
    A3 = tf.nn.relu(Z3)
    print("conv3 shape")
    print(conv3.shape)
    #pool3 = tf.nn.avg_pool(A3, ksize = [1,3,3,1], strides = [1,2,2,1], padding = 'VALID', data_format='NHWC', name=None)
    pool3 = tf.nn.max_pool(A3, ksize = [1,3,3,1], strides = [1,2,2,1], padding = 'VALID')
    print("pool3 shape before unfaltten")
    print(pool3.shape)
    pool3 = tf.contrib.layers.flatten(pool3)
    print("pool3 shape after flatten")
    print(pool3.shape)
    #fully connected layer relu as activation
    Z4 = tf.contrib.layers.fully_connected(pool3, 3000, activation_fn=None)
    A4 = tf.nn.relu(Z4)
    Z5 = tf.contrib.layers.fully_connected(A4, 400, activation_fn=None)
    #A5 = tf.nn.relu(Z5)
    A5 = tf.nn.relu(Z5)
    #dropout regularization
    #drop_out_A3 = tf.nn.dropout(A3, 0.95)
    Z6 = tf.contrib.layers.fully_connected(A5, 8, activation_fn=None)
    A6 = tf.nn.sigmoid(Z6)
    
    

    return A6

In [11]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001, iterations = 10, minibatch_size = 15):

    tf.set_random_seed(1)                                                                       
    (m, n_h, n_w, n_c) = X_train.shape             
    n_y = Y_train.shape[1]                            
    costs = [] 
    seed = 3
    weights, bias = initialize_weights()
    W1, W2, W3 = weights
    B1, B2, B3 = bias
    #print("weights intialized")
    #print(W1)
    X = tf.placeholder(tf.float32, shape=[None, n_h, n_w, n_c])
    Y = tf.placeholder(tf.float32, shape=[None, n_y])
    #forward propagation
    A5 = frwd_propagation(X, weights, bias)
    #calcualting cost
    cost_total = tf.nn.softmax_cross_entropy_with_logits(logits = A5, labels = Y)
    #l2 regularization
    #regularizer = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2)
    #cost = tf.reduce_mean(cost_total + 0.001 * regularizer)
    cost = tf.reduce_mean(cost_total )
    #back propagation
    adamoptimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    #soft = tf.nn.softmax(A5)
    init_op = tf.global_variables_initializer()
    saver = tf.train.Saver(tf.trainable_variables())
    
    with tf.Session() as sess:
        
        sess.run(init_op)
        #saver = tf.train.Saver([W1, W2, W3, B1, B2, B3])
        for iteration in range(iterations):
            
            print("iteration is %s" %iteration)
            #Weights1 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, 'conv1/kernel')[0]
            #Bias1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/bias')[0]
            #print("W1")
            #print(Weights1)
            cost_min_batch = 0
            seed = seed + 1
            
            minbatches = generate_min_batch(X_train, Y_train, seed, minibatch_size)
            minbatch_num = int(m / minibatch_size)
            for batch in minbatches:

                (min_batch_X, min_batch_Y) = batch
                _ , cost_single_batch = sess.run([adamoptimizer, cost], feed_dict={X: min_batch_X, Y: min_batch_Y})
                cost_min_batch += cost_single_batch / minbatch_num
            #_, c = sess.run([adamoptimizer, cost], feed_dict={X: X_train, Y: Y_train})
            print("iteration %s done" %iteration)  
            #gr = tf.get_default_graph()
            #conv1_kernel_val = gr.get_tensor_by_name('conv1/kernel:0').eval()
            #print("W1")
            #print(conv1_kernel_val)
            if iteration % 1 == 0:
                print("cost is %s " %cost_min_batch)
            
            #if iteration % 1 == 0:
                #costs.append(cost_min_batch) 
        print("iterations are done")
        

        # Calculate correct predictions
        predict_y = tf.argmax(A5, 1)
        #pred = tf.argmax(soft, 1)
        print("prediction y done")
        correct_prediction = tf.equal(predict_y, tf.argmax(Y, 1))
        print("corect pred computed")
        
        # Calculate accuracy on the TRAIN SET AND test set
        accuracy = 100 * tf.reduce_mean(tf.cast(correct_prediction, "float"))
        train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
        test_accuracy = accuracy.eval({X: X_test, Y: Y_test})
        #print("pred : %s" %predict_y.eval({X: X_test, Y: Y_test}))
        #print("Train Accuracy:", train_accuracy)
        #print("Test Accuracy:", test_accuracy)
        #new_y = predict_y.eval({X: X_train, Y: Y_train})
        #print(new_y)
        #saver.save(sess, 'my_test_model')
        #save_path = saver.save(sess, 'C:/Users/sahitya/258_assignment2/258_project/my_test_model')
        save_path = saver.save(sess, './my_test_model2')
        print("Model saved in path: %s" % save_path)
        #print(Weights1)       
        return train_accuracy, test_accuracy, weights, bias, costs

In [12]:
def generate_min_batch(X_train, Y_train, seed, minbatch_size):
    m = X_train.shape[0]
    #print("this is m %s" %m)
    np.random.seed(seed)
    random_p = np.random.permutation(m)
    #print("permutaion %s " %random_p)
    X_shuffled = X_train[random_p, :]
    Y_shuffled = Y_train[random_p, :]
    
    num_min_batches = m/minbatch_size
    min_batches = []
    for i in range(0, int(num_min_batches)):
        batch_X = X_shuffled[i * minbatch_size:(i + 1) * minbatch_size, :]
        batch_Y = Y_shuffled[i * minbatch_size:(i + 1) * minbatch_size, : ] 
        min_full_batch = (batch_X, batch_Y)
        min_batches.append(min_full_batch)
    
    return min_batches

In [13]:
from datetime import datetime
start=datetime.now()
train_accuracy, test_accuracy, weights, bias, costs = model(train_X, train_y_encode, test_x_norm1, test_y_encode)
print("train accuracy : %s" %train_accuracy)
print("test accuracy : %s" %test_accuracy)

end = datetime.now() - start

conv1 layer shape
(?, 256, 256, 6)
pool1 shape
(?, 127, 127, 6)
conv2 shape
(?, 123, 123, 8)
pool2 shape before unfaltten
(?, 61, 61, 8)
conv3 shape
(?, 61, 61, 10)
pool3 shape before unfaltten
(?, 30, 30, 10)
pool3 shape after flatten
(?, 9000)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

iteration is 0
iteration 0 done
cost is 1.9195772240559261 
iteration is 1
iteration 1 done
cost is 1.8367922912041352 
iteration is 2
iteration 2 done
cost is 1.7217806021372486 
iteration is 3
iteration 3 done
cost is 1.6049148162206015 
iteration is 4
iteration 4 done
cost is 1.54725806415081 
iteration is 5
iteration 5 done
cost is 1.5053022146224972 
iteration is 6
iteration 6 done
cost is 1.4673484623432154 
iteration is 7
iteration 7 done
cost is 1.4395515809456505 
iteration is 8
iteration 8 done
cost is 1.4153697441021604 
iteration is 9
iteration 9 